# Import modules

In [1]:
import torch, h5py, os
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# Load Data

In [2]:
# load dataset
dataset = h5py.File('/home/mohsen/Desktop/Academia/RUB Research Projects/INI/data/3dshapes/3dshapes.h5', 'r')
print(dataset.keys())
images = dataset['images']  # array shape [480000,64,64,3], uint8 in range(256)
labels = dataset['labels']  # array shape [480000,6], float64
image_shape = images.shape[1:]  # [64,64,3]
label_shape = labels.shape[1:]  # [6]
n_samples = labels.shape[0]  # 10*10*10*8*4*15=480000

<KeysViewHDF5 ['images', 'labels']>


In [46]:
labels[0]

array([  0.  ,   0.  ,   0.  ,   0.75,   0.  , -30.  ])

In [53]:
labels[0].shape

(6,)

In [ ]:
class LoadData(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        assert os.path.exists(self.data_path), "images path {} does not exist".format(self.data_path)
        self.data = h5py.File(self.data_path, 'r')
        self.images = self.data['images']  # array shape [480000,64,64,3], uint8 in range(256)
        self.labels = self.data['labels']  # array shape [480000,6], float64

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]

        # Normalize the image to range [0, 1]
        image = image.astype(np.float32) / 255.0
        # Transpose image to fit PyTorch's [C, H, W] format
        image = np.transpose(image, (2, 0, 1))

        if self.transform:
            image = self.transform(image)

        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.float32)

        return image, label
